# 4.1 金融风险管理 - 几何布朗运动模型模拟股票价格

本notebook演示了如何使用几何布朗运动(GBM)模型模拟股票价格路径，这是金融风险管理和资产定价中的基础模型。

## 关键概念解释：

* **几何布朗运动(Geometric Brownian Motion, GBM)**:
  * 描述随机变量的连续时间随机过程，广泛用于金融市场中建模股票价格变动
  * GBM假设股票收益率服从对数正态分布
  * 数学表示: $dS_t = \mu S_t dt + \sigma S_t dW_t$
    * $S_t$: 时间t的股票价格
    * $\mu$: 期望收益率(漂移项)
    * $\sigma$: 波动率
    * $dW_t$: 维纳过程(标准布朗运动)增量

* **离散化实现**:
  * 连续的GBM方程可以离散化为: $S_{t+\Delta t} = S_t \exp\left[(\mu-\frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t}Z_t\right]$
  * 其中$Z_t$是服从标准正态分布$N(0,1)$的随机变量
  * $\Delta t$是时间步长

* **应用价值**:
  * 风险评估：通过模拟未来价格路径，评估潜在风险敞口
  * 期权定价：Black-Scholes模型的基础假设
  * 投资组合分析：预测未来可能的市场情景

## 1. 安装必要的库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib

## 2. 下载数据文件

In [ ]:
# 2.1 从GitHub下载数据文件
import os

# 检查文件是否已存在，如果不存在则下载
if not os.path.exists("4.1 AAPL_data.csv"):
    print("从GitHub下载AAPL数据...")
    !wget -O "4.1 AAPL_data.csv" "https://raw.githubusercontent.com/Newzil-git/Financial-Risk-Management/main/FR%20Code/Part%204/4.1%20AAPL_data.csv"
else:
    print("AAPL数据文件已存在，跳过下载")

print("数据文件准备完成")

## 3. 导入相关库并设置中文显示

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 使用内建的中文字体设置
matplotlib.rcParams['font.family'] = 'SimHei'  
matplotlib.rcParams['axes.unicode_minus'] = False

## 4. 数据加载与预处理

加载历史股票数据并清理，用于估计模型参数。

In [ ]:
# 4.1 读取数据并查看前几行
try:
    data_file = "4.1 AAPL_data.csv"
    data = pd.read_csv(data_file)
    print("原始数据前5行:")
    display(data.head())
except Exception as e:
    print(f"读取数据文件时出错: {e}")
    print("创建模拟数据用于演示...")
    # 创建模拟数据用于演示
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', periods=252, freq='B')
    initial_price = 150
    prices = [initial_price]
    
    for _ in range(251):
        daily_return = np.random.normal(0.0005, 0.015)
        price = prices[-1] * (1 + daily_return)
        prices.append(price)
    
    data = pd.DataFrame({
        'Date': dates,
        'Close': prices,
        'High': [p * (1 + np.random.uniform(0, 0.02)) for p in prices],
        'Low': [p * (1 - np.random.uniform(0, 0.02)) for p in prices],
        'Open': [prices[i-1] if i > 0 else prices[0] for i in range(252)],
        'Volume': np.random.randint(5000000, 20000000, 252)
    })
    
    print("模拟数据前5行:")
    display(data.head())

In [ ]:
# 4.2 数据清理
# 将 'Close' 列转换为数字类型
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')

# 清理掉含有NaN的行
data_clean = data.dropna(subset=['Close'])
prices_clean = data_clean['Close'].values

print(f"清理后的数据点数量: {len(prices_clean)}")
prices_clean[:5]  # 显示前5个价格点

## 5. 参数估计

使用历史数据估计GBM模型的关键参数：漂移率(μ)和波动率(σ)。

In [ ]:
# 5.1 计算日收益率
returns_clean = np.diff(prices_clean) / prices_clean[:-1]

# 5.2 估计均值(mu)和波动率(sigma)
mu_clean = np.mean(returns_clean)
sigma_clean = np.std(returns_clean)

print(f"估计的日均收益率(μ): {mu_clean:.6f}")
print(f"估计的日波动率(σ): {sigma_clean:.6f}")
print(f"年化收益率: {mu_clean * 252:.4f}")
print(f"年化波动率: {sigma_clean * np.sqrt(252):.4f}")

## 6. 模拟设置

定义几何布朗运动模拟的参数和模拟函数。

In [ ]:
# 6.1 设置模拟参数
n_steps = 100  # 模拟的步数
delta_t = 1 / 252  # 每步的时间间隔（假设每年252个交易日）
initial_price = prices_clean[-1]  # 使用最后一个观察到的价格作为初始价格

print(f"模拟步数: {n_steps}")
print(f"时间步长: {delta_t:.6f}")
print(f"初始价格: {initial_price:.2f}")

In [ ]:
# 6.2 定义多路径模拟函数
def simulate_multiple_paths(n_paths, n_steps, mu, sigma, initial_price, delta_t):
    """
    使用几何布朗运动模型模拟多条股票价格路径
    
    参数:
    n_paths (int): 要模拟的路径数量
    n_steps (int): 每条路径的步数
    mu (float): 期望收益率(漂移项)
    sigma (float): 波动率
    initial_price (float): 初始股票价格
    delta_t (float): 时间步长
    
    返回:
    list: 包含多条模拟价格路径的列表
    """
    all_paths = []
    for _ in range(n_paths):
        simulated_prices = [initial_price]
        for i in range(n_steps):
            # 生成标准正态随机变量
            Z = np.random.normal(0, 1)
            # 应用GBM公式计算下一步价格
            next_price = simulated_prices[-1] * np.exp((mu - 0.5 * sigma ** 2) * delta_t + sigma * np.sqrt(delta_t) * Z)
            simulated_prices.append(next_price)
        all_paths.append(simulated_prices)
    return all_paths

## 7. 执行模拟

使用定义好的参数和函数执行股票价格路径模拟。

In [ ]:
# 7.1 模拟3条路径
n_paths = 3
paths = simulate_multiple_paths(n_paths, n_steps, mu_clean, sigma_clean, initial_price, delta_t)

# 7.2 显示模拟结果的前几个价格点
for i, path in enumerate(paths):
    print(f"路径 #{i+1} 的前5个价格点: {path[:5]}")

## 8. 可视化分析

绘制模拟的股票价格路径，以便直观地分析股票价格的可能变动。

In [ ]:
# 8.1 绘制多条路径
plt.figure(figsize=(12, 6))

for i, path in enumerate(paths):
    plt.plot(path, label=f'路径 #{i + 1}')

plt.title('几何布朗运动模拟的股票价格路径')
plt.xlabel('未来交易日')
plt.ylabel('股票价格')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 9. 生成模拟数据表格

将模拟结果保存为CSV文件，便于进一步分析。

In [ ]:
# 9.1 将模拟数据存储到DataFrame中
simulated_prices_clean = np.array(paths[0])  # 使用第一条路径
steps = list(range(n_steps + 1))

# 生成随机数和价格增量
uniform_random = [None] + [np.random.uniform(0, 1) for _ in range(n_steps)]
normal_random = [None] + [
    (mu_clean - 0.5 * sigma_clean ** 2) * delta_t + sigma_clean * np.sqrt(delta_t) * np.random.normal(0, 1) 
    for _ in range(n_steps)
]

# 创建DataFrame
simulated_data = pd.DataFrame({
    "Step": steps,
    "Uniform": uniform_random,
    "Normal": normal_random,
    "Price Increment": [None] + [
        (mu_clean - 0.5 * sigma_clean ** 2) * delta_t + sigma_clean * np.sqrt(delta_t) * np.random.normal(0, 1) 
        for _ in range(n_steps)
    ],
    "Price": simulated_prices_clean
})

# 显示模拟数据的前几行
simulated_data.head(10)

In [ ]:
# 9.2 保存模拟数据到CSV文件
simulated_data.to_csv('4.1_模拟.csv', index=False)
print("模拟数据已保存到 '4.1_模拟.csv'")

## 10. 风险管理应用

讨论几何布朗运动模型在金融风险管理中的应用。

### 10.1 风险度量计算

可以使用模拟的多条路径计算风险度量指标，如VaR（风险价值）和ES（期望损失）。

In [ ]:
# 10.1.1 模拟更多路径用于风险分析
n_paths_risk = 1000
forecast_horizon = 10  # 预测10天后的价格

# 生成多条路径
risk_paths = simulate_multiple_paths(
    n_paths_risk, forecast_horizon, mu_clean, sigma_clean, initial_price, delta_t
)

# 提取最终价格
final_prices = [path[forecast_horizon] for path in risk_paths]

# 计算收益率
returns = [(price - initial_price) / initial_price for price in final_prices]

# 计算95% VaR和ES
confidence_level = 0.95
var_95 = np.percentile(returns, (1 - confidence_level) * 100)
es_95 = np.mean([r for r in returns if r <= var_95])

print(f"基于{n_paths_risk}条路径的{forecast_horizon}天风险预测:")
print(f"95% VaR: {var_95*100:.2f}%")
print(f"95% ES: {es_95*100:.2f}%")
print(f"在金额上表示, 初始投资{initial_price:.2f}:")
print(f"95% VaR: {initial_price * abs(var_95):.2f}")
print(f"95% ES: {initial_price * abs(es_95):.2f}")

### 10.2 模型局限性分析

几何布朗运动模型虽然广泛应用，但也存在一些局限性：

1. **波动率假设**：
   - GBM假设波动率恒定，而实际市场中波动率往往会随时间变化（波动率聚集现象）
   - 市场压力期间，波动率通常会突然增加

2. **收益率分布**：
   - 真实市场收益率通常表现出更厚的尾部分布，而非GBM假设的正态分布
   - 极端事件（市场崩盘）发生的频率高于模型预测

3. **连续路径**：
   - GBM产生连续的价格路径，而实际市场中存在跳跃和缺口
   - 无法捕捉隔夜风险或极端价格变动

4. **独立性假设**：
   - 模型假设价格变动相互独立，忽略了市场可能存在的自相关性和趋势

尽管有这些局限性，GBM仍然是金融风险管理中最基础、应用最广泛的模型之一，为更复杂的模型提供了基础。